## --- Day 3: Gear Ratios ---
You and the Elf eventually reach a gondola lift station; he says the gondola lift will take you up to the water source, but this is as far as he can bring you. You go inside.

It doesn't take long to find the gondolas, but there seems to be a problem: they're not moving.

"Aaah!"

You turn around to see a slightly-greasy Elf with a wrench and a look of surprise. "Sorry, I wasn't expecting anyone! The gondola lift isn't working right now; it'll still be a while before I can fix it." You offer to help.

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:
``` 
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
``` 
In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?

In [12]:
#make boolean mask for array as solutioim
import numpy as np
from scipy.ndimage import convolve

In [13]:
test_schematic = """467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598.."""

test_schematic = test_schematic.split("\n")

test_schematic = np.array([[x for x in l] for l in test_schematic])

test_schematic

array([['4', '6', '7', '.', '.', '1', '1', '4', '.', '.'],
       ['.', '.', '.', '*', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '3', '5', '.', '.', '6', '3', '3', '.'],
       ['.', '.', '.', '.', '.', '.', '#', '.', '.', '.'],
       ['6', '1', '7', '*', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '+', '.', '5', '8', '.'],
       ['.', '.', '5', '9', '2', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '7', '5', '5', '.'],
       ['.', '.', '.', '$', '.', '*', '.', '.', '.', '.'],
       ['.', '6', '6', '4', '.', '5', '9', '8', '.', '.']], dtype='<U1')

In [14]:
with open("./input.txt", "r") as f:
    schematic = f.readlines()
    schematic = [l.rstrip("\n") for l in schematic]
    schematic = np.array([[x for x in s] for s in schematic])

In [15]:
schematic

array([['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '.', '3', ..., '.', '.', '.'],
       ...,
       ['.', '.', '.', ..., '2', '4', '9'],
       ['.', '.', '.', ..., '.', '.', '.'],
       ['.', '1', '2', ..., '.', '.', '.']], dtype='<U1')

In [16]:
#all the chars that are not symbols to check against
not_symbols = "1234567890."

In [17]:

def create_symbol_map(schematic):
    #initialize first mask to store symbol locations
    symbol_location_map = np.ones(schematic.shape)

    #set every location that is not a symbol to 0
    w, h = schematic.shape
    for i in range(w):
        for j in range(h):
            symbol_location_map[i,j] = schematic[i,j] not in not_symbols
    return symbol_location_map


In [18]:
def create_valid_coordinate_map(symbol_location_map, kernel = np.array([[1,1,1],[1,0,1],[1,1,1]])):
    valid_coordinate_map = convolve(symbol_location_map, kernel, mode = "constant", cval = 0)
    valid_coordinate_map = valid_coordinate_map.astype(int)
    return valid_coordinate_map

`symbol_location_map`  now has ones in all locations a symbol is. Now we need to set a pixel in a new mask to 1 if the surrounding pixels contain a 1. We can do a convolution with a kernel that just sets the pixel to positive if htere is anythiing positive surrounding it, else 0. 
The kernel will look like :


In [19]:
def create_number_map(schematic):
    #we need a map of where the numbers are too:
    w, h = schematic.shape
    number_map = np.zeros(schematic.shape)
    for i in range(w):
        for j in range(h):
            number_map[i,j] = schematic[i,j].isnumeric()
    number_map = number_map.astype(bool)
    return number_map

In [20]:
#coordinates of all valid digits. If a number has a digit in it on one of these coordinates, we count the number
coordinates = np.where(valid_coordinate_map)

NameError: name 'valid_coordinate_map' is not defined

Now we have to gather all the numbers and their coordinates in some smart structure to do the checks against the coordinates. One line of the schematic can have multiple numbers. We can have a stupid double-loop with a flag type solution to get us going:

In [21]:
def find_groups(arr):
    first = arr[0]
    last = arr[0]
    for x in arr[1:]:
        if x == last+1:
            last = x
        else:
            yield first, last
            first = x
            last = x
    yield first, last

In [22]:
def find_sum_of_parts(schematic, number_map, valid_coordinates, debug = False):
    valid_numbers = []
    w, h = schematic.shape
    coordinates = np.where(valid_coordinates)[0]
    for x in range(w):
        valid_row_numbers = []
        if x in coordinates[0]:
            number_locs = np.where(number_map[x])[0]
            if len(number_locs) > 0:
                number_groups = [np.arange(group[0], group[-1]+1) if group[0]<group[-1]+1 else np.array([group[0]]) for group in find_groups(number_locs)]
                # print(number_groups)
                for group in number_groups:
                    # print(x, group, valid_coordinates[x, group])
                    if sum(valid_coordinates[x, group]) > 0:
                            valid_row_number = int("".join(schematic[x, group[0]: group[-1]+1]))
                            valid_numbers.append(valid_row_number)
    if debug:
        print(valid_numbers)
    return sum(valid_numbers)

                
                
                

In [23]:
def find_numbers(schematic, debug = False):
    symbol_map = create_symbol_map(schematic)
    number_map = create_number_map(schematic)
    valid_coordinates = create_valid_coordinate_map(symbol_map)
    # valid_number_map = number_map&valid_coordinates
    return find_sum_of_parts(schematic, number_map, valid_coordinates, debug)


In [24]:
find_numbers(schematic)

TypeError: argument of type 'numpy.int64' is not iterable

The engineer finds the missing part and installs it in the engine! As the engine springs to life, you jump in the closest gondola, finally ready to ascend to the water source.

You don't seem to be going very fast, though. Maybe something is still wrong? Fortunately, the gondola has a phone labeled "help", so you pick it up and the engineer answers.

Before you can explain the situation, she suggests that you look out the window. There stands the engineer, holding a phone in one hand and waving with the other. You're going so slowly that you haven't even left the station. You exit the gondola.

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:
``` 
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
``` 
In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?

For this we need to maybe do another convolution? Or solve the problem in an entirely different way?

In [25]:
potential_gear_map = test_schematic == "*"

In [26]:
potential_gear_map.astype(int)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [27]:
potential_gears = np.argwhere(potential_gear_map)

In [28]:
potential_gears

array([[1, 3],
       [4, 3],
       [8, 5]])

In [58]:
def find_valid_number_map(schematic, number_map, valid_coordinates, debug = False):
    valid_numbers = []
    valid_number_map = np.zeros(schematic.shape)
    coordinates = np.where(valid_coordinates)
    w, h = schematic.shape
    for x in range(w):
        valid_row_numbers = []
        if x in coordinates[0]:
            number_locs = np.where(number_map[x])[0]
            if len(number_locs) > 0:
                number_groups = [np.arange(group[0], group[-1]+1) if group[0]<group[-1]+1 else np.array([group[0]]) for group in find_groups(number_locs)]
                # print(number_groups)
                for group in number_groups:
                    # print(x, group, valid_coordinates[x, group])
                    if sum(valid_coordinates[x, group]) > 0:
                        valid_number_map[x, group] = 1
                        valid_row_number = int("".join(schematic[x, group[0]: group[-1]+1]))
                        valid_numbers.append(valid_row_number)
    if debug:
        print(valid_numbers)
    return valid_number_map

In [59]:
def find_valid_numbers(schematic, debug = False):
    symbol_map = create_symbol_map(schematic)
    number_map = create_number_map(schematic)
    valid_coordinates = create_valid_coordinate_map(symbol_map)
    # valid_number_map = number_map&valid_coordinates
    valid_number_map = find_valid_number_map(schematic, number_map, valid_coordinates, debug)
    return valid_number_map.astype(int)


In [60]:
def drop_consecutives(block):
    for row in block:
        for i in range(len(row)-1):
            if row[i] == 1 and row[i+1] == 1:
                row[i] = 0
    return block

In [61]:
def is_gear(x, y, valid_digit_map):
    neighbours = drop_consecutives(valid_digit_map[x-1:x+2, y-1:y+2])
    if np.sum(neighbours) == 2:
        

In [62]:
def find_gear_map(schematic):
    gear_map = np.zeros(schematic.shape)
    potential_gears = np.argwhere(schematic == "*")
    valid_digit_map = find_valid_numbers(schematic)
    for gear in potential_gears:
        x, y = gear
        if is_gear(x, y, valid_digit_map):
            gear_map[x,y] = 1
    return gear_map
    

In [63]:
def drop_contiguous_points(coords):
    contiguous = np.all(
        np.logical_or(
            coords[1:] == coords[:-1], #any coordinate match forwards
            coords[1:] == coords[:-1] + 1, #any coordinate match reverse
        ),
        axis=1,
    )
    mask = np.append([True], np.logical_not(contiguous)) #we want the not-contiguous ones
    return coords[mask]


In [64]:
def scrape_number(x, y, schematic):
    number_map = np.char.isdigit(schematic)
    number = ""
    i = y
    while i >= 0 and number_map[x, i]:
        number = schematic[x, i]+number
        i-=1
    i=y+1
    while i<len(schematic[x]) and number_map[x, i]:
        number = number + schematic[x, i]
        i+=1
    return(int(number))
    

In [67]:
def find_gear_ratios(test_schematic):
    #map all valid gears
    gear_map = find_gear_map(test_schematic)
    print(gear_map)
    #coordinates of all valid gears
    gear_locations = np.argwhere(gear_map)

    #map of all locations of valid gear neighbours
    valid_gear_neigbour_locations = create_valid_coordinate_map(gear_map)
    number_map = np.char.isdigit(test_schematic)
    valid_gear_neighbours = valid_gear_neigbour_locations & number_map
    print(valid_gear_neighbours)
    #coordinates with doubles dropped
    coords = np.argwhere(valid_gear_neighbours)
    print(coords)
    coords = drop_contiguous_points(coords)

    numbers = np.array([scrape_number(coord[0], coord[1], test_schematic) for coord in coords])
    numbers = numbers.reshape(len(numbers)//2, 2)

    return(np.sum(np.prod(numbers, axis = 1)))

In [73]:
valid_gear_neigbour_locations = create_valid_coordinate_map(gear_map)
number_map = np.char.isdigit(test_schematic)
valid_gear_neighbours = valid_gear_neigbour_locations & number_map
print(valid_gear_neighbours)

[[0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 1 0 0 0]]


In [109]:
def find_gear_ratios2(test_schematic):
    gear_map = find_gear_map(test_schematic)
    gear_locations = np.argwhere(gear_map)
    # print(f"{test_schematic=}\n")
    # print(f"{gear_map=}\n")
    products = []
    for gear in gear_locations:
        x, y = gear
        context = test_schematic[x-1:x+2, y-1:y+2]
        coords = np.argwhere(np.char.isdigit(context))
        single_coords = drop_contiguous_points(coords)
        translated_coords = single_coords + gear -np.array([1,1])
        numbers = [scrape_number(coord[0], coord[1], test_schematic) for coord in translated_coords]
        product = np.product(numbers)
        products.append(product)
        # print(f"{gear=}\n {context}\n {single_coords=}\n {translated_coords=}\n {numbers=}\n\n")
    return(sum(products))

In [110]:
find_gear_ratios2(test_schematic)

467835

In [111]:
find_gear_ratios2(schematic)

72246648

In [76]:
drop_contiguous_points(np.argwhere(valid_gear_neighbours))

array([[0, 2],
       [2, 2],
       [7, 6],
       [9, 5]])

In [72]:
np.argwhere(gear_map)

array([[1, 3],
       [8, 5]])